In [5]:
import os 
import sys
import datetime
import csv 

import cirq
import qsimcirq
import openfermion as of
import numpy as np
from scipy.sparse import linalg
import multiprocessing as mp
Pi=3.1415


$$ \begin{align} H = &- t \sum_{\langle i,j \rangle} \sum_{\sigma} (a^\dagger_{i, \sigma} a_{j, \sigma} + a^\dagger_{j, \sigma} a_{i, \sigma})


     + U \sum_{i} a^\dagger_{i, \uparrow} a_{i, \uparrow}
                 a^\dagger_{i, \downarrow} a_{i, \downarrow}
    \\
    &- \mu \sum_i \sum_{\sigma} a^\dagger_{i, \sigma} a_{i, \sigma}
     - h \sum_i (a^\dagger_{i, \uparrow} a_{i, \uparrow} -
               a^\dagger_{i, \downarrow} a_{i, \downarrow})
\end{align}
$$

$$
    U \sum_{k=1}^{N-1} a_k^\dagger a_k a_{k+1}^\dagger a_{k+1}
$$


If True, the repulsion term is replaced by:


$$
    U \sum_{k=1}^{N-1} (a_k^\dagger a_k - \frac12)
                       (a_{k+1}^\dagger a_{k+1} - \frac12)
$$


which is unchanged under a particle-hole transformation.
Default is False


In [3]:
class HubbardArgs():
    def __init__(self, 
                x_dimension,
                y_dimension,
                tunneling,
                coulomb,
                chemical_potential=0.0,
                magnetic_field=0.0,
                periodic=True,
                spinless=False,
                particle_hole_symmetry=False,
                qsim_option=None):
        
        self.x_dimension = x_dimension
        self.y_dimension = y_dimension
        self.tunneling = tunneling
        self.coulomb = coulomb
        
        self.chemical_potential = chemical_potential
        self.magnetic_field = magnetic_field
        self.periodic = periodic
        self.spinless = spinless
        self.particle_hole_symmetry = particle_hole_symmetry

        if not (qsim_option):
            self.qsim_option = qsim_option
        else:
            qsim_option = {'t': 4, 'f': 1}


In [54]:
def exact_eigenvalue( 
                x_dimension=1,
                y_dimension=4,
                tunneling=1,
                coulomb=4,
                chemical_potential=0.0,
                magnetic_field=0.0,
                periodic=True,
                spinless=False,
                particle_hole_symmetry=False,
                qsim_option=None, 
                particle_n=4):
    function_args = HubbardArgs( 
                    x_dimension,
                    y_dimension,
                    tunneling,
                    coulomb,
                    chemical_potential,
                    magnetic_field,
                    periodic,
                    spinless,
                    particle_hole_symmetry,
                    qsim_option)

    # define a Hubbard hamiltonian and convert into qubit operator through jordan-wigner transformation
    fermion_hamiltonian = of.fermi_hubbard(
        function_args.x_dimension,
        function_args.y_dimension,
        function_args.tunneling,
        function_args.coulomb,
        function_args.chemical_potential,
        function_args.magnetic_field,
        function_args.periodic,
        function_args.spinless,
        function_args.particle_hole_symmetry
    )
    qubit_hamiltonian = of.transforms.jordan_wigner(fermion_hamiltonian)

    # Convert to Scipy sparse matrix
    hamiltonian_jw_sparse = of.get_sparse_operator(qubit_hamiltonian)

    # Compute ground energy
    eigs, _ = linalg.eigsh(hamiltonian_jw_sparse, k=1, which="SA")
    ground_energy = eigs[0]
    # print("Ground_energy: {}".format(ground_energy))
    # print("JWT transformed Hamiltonian:")
    # print(qubit_hamiltonian)

    energy_on_n = of.linalg.jw_get_ground_state_at_particle_number(
        hamiltonian_jw_sparse, particle_n
    )
    
    # print(energy_on_n[0])
    return ground_energy, energy_on_n[0]



check if energy values equal to its exact one

Ut=2and4, on half-filling
- [x] W=1 L=2 
- [x] W=1 L=4 
- [x] W=1 L=8
- [x] W=2 L=2
- [ ] W=2 L=4


In [55]:
print("x y n c energy, energy_on_n")
for x in [1]:
    for y in [2]:
        for n in [1,2]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n
                ))

x y n c energy, energy_on_n
1 2 1 2 (-1.2360679774997891, -1.0000000000000004)
1 2 1 4 (-1.0000000000000013, -0.9999999999999999)
1 2 2 2 (-1.2360679774997902, -1.2360679774997894)
1 2 2 4 (-1.000000000000002, -0.8284271247461907)


In [56]:
print("x y n c energy, energy_on_n")
for x in [1]:
    for y in [4]:
        for n in [2,4,8]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n
                ))

x y n c energy, energy_on_n
1 4 2 2 (-3.6272130052966616, -3.627213005296653)
1 4 2 4 (-3.418550718873847, -3.4185507188738455)
1 4 4 2 (-3.627213005296658, -2.828427124746192)
1 4 4 4 (-3.418550718873843, -2.102748483462073)
1 4 8 2 (-3.627213005296665, 8.0)
1 4 8 4 (-3.418550718873837, 16.0)


In [49]:
print("x y n c energy, energy_on_n")
for x in [1]:
    for y in [8]:
        for n in [2,4,8]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n
                ))

x y n c energy, energy_on_n
1 8 2 2 (-7.826311280176336, -3.854940367646751)
1 8 2 4 (-6.672195997058454, -3.8005393112446404)
1 8 4 2 (-7.8263112801763715, -6.261566147504012)
1 8 4 4 (-6.672195997058458, -5.951702657355303)
1 8 8 2 (-7.826311280176379, -6.568192162874705)
1 8 8 4 (-6.672195997058454, -4.6035262999891975)


In [57]:
print("x y n c energy, energy_on_n")
for x in [2]:
    for y in [2,4]:
        for n in [4,8]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n
                ))

x y n c energy, energy_on_n
2 2 4 2 (-3.627213005296674, -2.8284271247461907)
2 2 4 4 (-3.4185507188738473, -2.1027484834620696)
2 2 8 2 (-3.627213005296672, 8.0)
2 2 8 4 (-3.4185507188738504, 16.0)
2 4 4 2 (-8.47830329686955, -7.431230836069724)
2 4 4 4 (-7.6124749277658745, -7.0922664292384585)
2 4 8 2 (-8.478303296869584, -8.478303296869631)
2 4 8 4 (-7.612474927765864, -5.954236681057309)


In [58]:
print("x y n c energy, energy_on_n")
for x in [2]:
    for y in [2,4]:
        for n in [4,8]:
            for coulomb in [2,4]:
                print(x, y, n, coulomb, exact_eigenvalue(
                    x_dimension=x, y_dimension=y,coulomb=coulomb, particle_n=n,
                    periodic=False
                ))

x y n c energy, energy_on_n
2 2 4 2 (-3.627213005296672, -2.8284271247461907)
2 2 4 4 (-3.4185507188738415, -2.102748483462075)
2 2 8 2 (-3.627213005296659, 8.0)
2 2 8 4 (-3.4185507188738473, 16.0)
2 4 4 2 (-7.912602325322799, -7.713287732110458)
2 4 4 4 (-7.2744870134962625, -7.274487013496239)
2 4 8 2 (-7.912602325322815, -7.117446580313431)
2 4 8 4 (-7.2744870134962305, -5.012503152657044)


In [ ]:
# circuit hubbard

In [ ]:
# bcs state circuit

function_args = HubbardArgs( 
                x_dimension,
                y_dimension,
                tunneling,
                coulomb,
                chemical_potential,
                magnetic_field,
                periodic,
                spinless,
                particle_hole_symmetry,
                qsim_option)

mean_field_model = of.mean_field_dwave(
    x_dimension, y_dimension, tunneling, sc_gap, periodic)
quad_ham = of.get_quadratic_hamiltonian(mean_field_model)


_, basis_change_matrix, _ = quad_ham.diagonalizing_bogoliubov_transform()
qubits = cirq.LineQubit.range(n_qubits)
circuit = cirq.Circuit(of.circuits.bogoliubov_transform(qubits, basis_change_matrix))


In [ ]:
# bcs e^(-iHt) circuit


In [ ]:
# hubbard e^(-iHt) circuit


In [ ]:
# anzats hubbard

# for layer in range(p)

    # H1 bcs 

    # H2 hubbard



In [ ]:
# expectation hubbard

In [ ]:
# optimize